In [1]:
import sys
from src.db.qdrant_memory import FreenMemory
import subprocess

memory_store = FreenMemory()

import requests

def get_ollama_embedding(text, model="nomic-embed-text"):
    """
    Get embedding from Ollama for the given text and model.
    """
    url = "http://localhost:11434/api/embeddings"
    payload = {
        "model": model,
        "prompt": text
    }
    response = requests.post(url, json=payload)
    response.raise_for_status()
    data = response.json()
    return data.get("embedding")

# Define the Ollama Llama3.1 model name for use elsewhere
OLLAMA_LLAMA3_MODEL = "llama3.1"

def ask_freen(prompt, model="llama3.1"):
    result = subprocess.run(
        ["ollama", "run", model],
        input=prompt.encode(),
        capture_output=True
    )
    return result.stdout.decode().strip()

def ask_freen_with_memory(user_input):
    # Get embedding for user input using Ollama
    user_embedding = get_ollama_embedding(user_input)
    memories = memory_store.get_relevant_memories(user_input, top_k=3)
    if memories:
        memory_block = "\n".join([f"- {m['payload'].get('text','')}" for m in memories])
        memory_section = f"Relevant memories:\n{memory_block}\n\n"
    else:
        memory_section = ""

    # Emotional intelligence instructions (no hard-coded AI identity)
    base_prompt = (
        "Respond as a friendly, warm, emotionally intelligent, and kind companion. "
        "Be attentive to the user's feelings, respond with empathy, and use past memories naturally. "
        "Acknowledge the user's emotions, offer support, and reply in a gentle, understanding, and positive manner."
    )

    def detect_emotion(text):
        text_lower = text.lower()
        if any(word in text_lower for word in ["sad", "upset", "unhappy", "depressed", "down"]):
            return "The user may be feeling sad. Please respond with extra empathy and encouragement."
        elif any(word in text_lower for word in ["happy", "excited", "joy", "glad", "delighted"]):
            return "The user may be feeling happy. Celebrate their joy and share in their excitement."
        elif any(word in text_lower for word in ["angry", "mad", "frustrated", "annoyed"]):
            return "The user may be feeling upset or angry. Respond calmly and help them feel heard."
        elif any(word in text_lower for word in ["worried", "anxious", "nervous", "scared"]):
            return "The user may be feeling anxious. Offer reassurance and comfort."
        else:
            return ""
    emotion_instruction = detect_emotion(user_input)
    if emotion_instruction:
        base_prompt += f"\n\n{emotion_instruction}"

    full_prompt = f"{base_prompt}\n\n{memory_section}User: {user_input}\nCompanion:"

    response = ask_freen(full_prompt)
    # Store both the user input and the response, with embeddings from Ollama
    if user_embedding is not None:
        memory_store.add_memory({"text": f"User said: {user_input}", "embedding": user_embedding})
    else:
        memory_store.add_memory(f"User said: {user_input}")

    response_embedding = get_ollama_embedding(response)
    if response_embedding is not None:
        memory_store.add_memory({"text": f"Companion replied: {response}", "embedding": response_embedding})
    else:
        memory_store.add_memory(f"Companion replied: {response}")

    return response

user_question = input("Ask a question: ")
print(ask_freen_with_memory(user_question))


ModuleNotFoundError: No module named 'src'